In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
list_url

总数量：6586


['https://www.1aauto.com/catalog/product/fit?skuId=1218703&pathName=%2F1996-00-chevrolet-k3500-gmc-k3500-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29768%2Fi%2F1abfs16341',
 'https://www.1aauto.com/catalog/product/fit?skuId=1179366&pathName=%2F1990-02-chevrolet-astro-gmc-safari-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka18389%2Fi%2F1abfs08752',
 'https://www.1aauto.com/catalog/product/fit?skuId=1179945&pathName=%2F1998-02-chevrolet-prizm-toyota-corolla-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka18480%2Fi%2F1abfs08698',
 'https://www.1aauto.com/catalog/product/fit?skuId=1219746&pathName=%2F1996-00-chevrolet-k3500-gmc-k3500-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29767%2Fi%2F1abfs16340',
 'https://www.1aauto.com/catalog/product/fit?skuId=1177994&pathName=%2F1990-02-chevrolet-astro-gmc-safari-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka18434%2Fi%2F1abfs08776',
 'https://

In [3]:
df = pd.DataFrame()

df

""


In [4]:
import requests

In [5]:
i = 0

url = list_url[i]

resp = requests.get(url).text

resp

'<div id="product-fit-tab">\n\n    <hr class="hr">\n    <div id="available-makes" class="available-makes">\n        \n        <h2>Fits these Makes</h2>\n        <div id="available-makes-list" class="available-makes-list expanderContainer">\n            <div id="fit-makes-filter" class="flex-box fit-makes-filter filter expanderContainerWrapper "\n                 data-filter-target=".fitTable">\n                \n                    <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Chevrolet">Chevrolet\n                    </button>\n                \n                    <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="GMC">GMC\n                    </button>\n                \n            </div>\n            <div class="expander-container-button">\n                <button class="expanderSwitch closed expander-switch btn btn-link d-none"\n                        data-expander-container="#fit-makes-filter">view more\n   

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(resp, 'lxml')

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div id="product-fit-tab">
   <hr class="hr"/>
   <div class="available-makes" id="available-makes">
    <h2>
     Fits these Makes
    </h2>
    <div class="available-makes-list expanderContainer" id="available-makes-list">
     <div class="flex-box fit-makes-filter filter expanderContainerWrapper" data-filter-target=".fitTable" id="fit-makes-filter">
      <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="Chevrolet">
       Chevrolet
      </button>
      <button class="btn btn-link fit-make-filter-button filterOption" data-filter-value="GMC">
       GMC
      </button>
     </div>
     <div class="expander-container-button">
      <button class="expanderSwitch closed expander-switch btn btn-link d-none" data-expander-container="#fit-makes-filter">
       view more
      </button>
     </div>
    </div>
   </div>
   <div class="vehicle-fit cf">
    <div class="fitTable-area">
     <input class="isTableSorted" type="hidden" value="fal

In [8]:
from lxml import etree

In [23]:
html = etree.HTML(str(soup))

list_tr = html.xpath('//tr[@class="sortable_row filter-field"]')

list_tr

[<Element tr at 0x1277b5a00>,
 <Element tr at 0x127714e40>,
 <Element tr at 0x127716700>,
 <Element tr at 0x127714100>,
 <Element tr at 0x127716240>,
 <Element tr at 0x127716180>]

In [24]:
dict_vehicle = {}
for tr in list_tr:
    make = tr.xpath('./td[2]/text()')[0].strip()
    model = tr.xpath('./td[3]/text()')[0].strip()
    year_begin = int(tr.xpath('./td[1]/text()')[0].split('-')[0].strip())
    year_end = int(tr.xpath('./td[1]/text()')[0].split('-')[-1].strip())
    
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: [year_begin, year_end]}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year_begin, year_end]
        else:
            dict_vehicle[make][model].append(year_begin)
            dict_vehicle[make][model].append(year_end)

dict_vehicle

{'Chevrolet': {'K3500 Truck': [1996, 2000, 1996, 2000, 1996, 2000]},
 'GMC': {'K3500 Truck': [1996, 2000, 1996, 2000, 1996, 2000]}}

In [25]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        year_max = max(dict_vehicle[make][model])
        year_min = min(dict_vehicle[make][model])
        if year_max == year_min:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

vehicle = '\n'.join(list_vehicle)

print(vehicle)

Chevrolet K3500 Truck 1996-2000
GMC K3500 Truck 1996-2000


In [30]:
list_tr = html.xpath('//table[@id="interchange-codes-table"]/tbody/tr')

list_tr

[<Element tr at 0x12768e540>,
 <Element tr at 0x12768f700>,
 <Element tr at 0x12768fa80>,
 <Element tr at 0x12768e800>,
 <Element tr at 0x12768f7c0>,
 <Element tr at 0x12768e580>,
 <Element tr at 0x12768fbc0>,
 <Element tr at 0x12768fe00>,
 <Element tr at 0x12768e940>,
 <Element tr at 0x12768e040>,
 <Element tr at 0x12768fa00>,
 <Element tr at 0x12768c180>,
 <Element tr at 0x12768d240>,
 <Element tr at 0x12768e280>,
 <Element tr at 0x12768f480>,
 <Element tr at 0x12768cc40>,
 <Element tr at 0x12768c7c0>,
 <Element tr at 0x12768cf00>,
 <Element tr at 0x12768d000>,
 <Element tr at 0x12768c940>,
 <Element tr at 0x12768c800>,
 <Element tr at 0x127d63080>,
 <Element tr at 0x127d626c0>,
 <Element tr at 0x1276c5080>,
 <Element tr at 0x1276c4200>,
 <Element tr at 0x1276c4ac0>,
 <Element tr at 0x1276c5540>,
 <Element tr at 0x1276c7180>,
 <Element tr at 0x1276c4540>,
 <Element tr at 0x123c7c9c0>,
 <Element tr at 0x123c7fa00>,
 <Element tr at 0x127dc63c0>,
 <Element tr at 0x127dc7140>,
 <Element 

In [31]:
list_oe = []
for tr in list_tr:
    list_oe.append(tr.xpath('./td[2]/text()')[0].strip())
    
list_oe = list(set(list_oe))
list_oe.sort()

oe = ';'.join(list_oe)

oe

'04886279AB;10-C4012;10-C4013;12321454;14-4012;14-4013;141.66021;141.66022;150.66072;150.66073;18-4347;18-4348;18024913;18045608;18FR981C;18FR982C;18J1647;18J1648;19167290;19242609;22-17271L;22-17271R;2201181;2203179;22163451;22163452;331-40074;331-40075;350-47284;350-47285;4746352;55028;56324;78403;78460;99-17271L-C;99-17271R-C;BD125346;BH138063;BH138064;BH380533;BH380534;BR55028;C66021;C66022;EHT459H;FRC10183;FRC10183N;FRC10184;FRC10184N;H380533;H380534;MD459;PGD459M;PRT1770'

In [32]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'Vehicle_Url': url,
                         'Vehicle': vehicle,
                         'OE': oe}])

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,status,Vehicle_Url,Vehicle,OE
0,ok,https://www.1aauto.com/catalog/product/fit?sku...,Chevrolet K3500 Truck 1996-2000\nGMC K3500 Tru...,04886279AB;10-C4012;10-C4013;12321454;14-4012;...
